# Seagrass mapping in two mudflats in the Auray River

About a rapid evolution of seagrasses

Simon Oiry [](https://orcid.org/0000-0001-7161-5246) (Nantes Université, UR 2160, F-44000 Nantes, France)  
Bede Ffinian Rowe Davies [](https://orcid.org/0000-0001-6462-4347) (Nantes Université, UR 2160, F-44000 Nantes, France)  
July 17, 2024

Maps of seagrass in two sites in the Auray River. These two sites were studied by Maxime Daviray during his PhD. Seagrass appeared very quickly during his PhD. This work aims to describe this rapid evolution of seagrasses.

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attachement du package : 'tidyterra'

L'objet suivant est masqué depuis 'package:stats':

    filter

terra 1.7.78

Attachement du package : 'terra'

L'objet suivant est masqué depuis 'package:tidyr':

    extract


Attachement du package : 'patchwork'

L'objet suivant est masqué depuis 'package:terra':

    area

The data and scripts used for this work can be found [here](https://github.com/SigOiry/Seagrass_maps_Maxime).

# Materials & Methods

## Seagrass mapping using Sentinel-2

To map the seagrass extent over time, the Sentinel-2 constellation has been used. Level-2 images, which are already orthorectified and atmospherically corrected using Sen2Cor, have been downloaded using the Copernicus Platform \[@Copernicus_Sentinel\]. One low tide, cloud-free image per year, nearest to the period of maximum seagrass biomass at this latitude, has been used. A total of 8 images have beed used (@tbl-tide-data).

| Acquisition Date (UTC) | Low Tide Time (UTC) | Time Difference with Low tide |
|:---------------------:|:------------------:|:---------------------------:|
|    2016-11-03 11:12    |       12 : 08       |          \+ 00 : 56           |
|    2017-10-04 11:08    |       09 : 09       |          \- 01 : 59           |
|    2018-09-29 11:08    |       12 : 43       |          \+ 01 : 35           |
|    2019-09-14 11:06    |       10 : 28       |          \+ 00 : 38           |
|    2020-08-04 11:06    |       10 : 45       |          \+ 00 : 21           |
|    2021-10-08 11:09    |       11 : 18       |          \- 00 : 09           |
|    2022-08-29 11:06    |       11 : 27       |          \- 00 : 21           |
|    2023-09-03 11:06    |       12 : 28       |          \- 01 : 22           |

The *Intertidal Classification of Europe: Categorising Reflectance of Emerged Areas of Marine vegetation with Sentinel-2* model (ICE CREAMS, @Davies2024), a neural network classifier designed to identify and discriminate intertidal vegetation in Europe, has been applied to each Sentinel-2 image. Pixels of the Magnoliopsida class (seagrasses) have been isolated, and the Normalized Difference Vegetation Index (NDVI, @rouse1974monitoring), a commonly used remote sensing biomass proxy for vegetation, has been employed. The equation of @zoffoli2020sentinel have been used to transform NDVI values into Seagrass Percent Cover (\[@eq-std\]). SPC values below 20%, corresponding to low biomass pixel, with a high risk of confusion with other vegetation classes, have been remove from the rest of the analysis.

$$
SPC = 172.06 \times NDVI - 22.18
$$ {#eq-std}

Maps and analysis have then been performed using the *Terra* package of R, in a *Tidyverse* workflow \[@hijmans2023terra ; @wickham2017easily\].

# Results

## Evolution of the spatial distribution of seagrasses over time

In [ ]:
rgb_list <- list.files("Data/RGB/", pattern = ".tif", recursive = TRUE, full.names = TRUE) %>%
  as_tibble() %>% 
  rename(path_rgb = "value") %>%  
  mutate(name = gsub(".*/","",path_rgb),                 # Extract the filename
         year = substr(name,1,4)) %>% 
  dplyr::select(-name)

img_list <- list.files("Data/ICECREAMS/", pattern = ".tif", recursive = TRUE, full.names = TRUE) %>% 
  as_tibble() %>% 
  rename(path_pred = "value") %>% 
  mutate(name = gsub(".*/","",path_pred),                 # Extract the filename
         year = substr(name,12,15),                  # Extract the year from the filename
         date = as.POSIXct(substr(name,12,19), format = "%Y%m%d")) %>% 
  left_join(rgb_list, by = "year")



# Extract and format the date
mask <- "Data/mask/Intertidal_mask_Auray.shp" %>% 
  vect()

for (i in 1:nrow(img_list)) {
  
  pred <- img_list$path_pred[i] %>% 
    rast(lyrs = 5) %>% 
    mask(mask) %>% 
    as.data.frame(xy=T) %>% 
    mutate(SPC20Unknown = case_when(SPC20Unknown < 20 ~ NA,
                     T ~ SPC20Unknown))
    
  rgb <- img_list$path_rgb[i] %>% 
    rast()

    for(site_i in 1:length(unique(mask$Site_Name))){
    
      # Extract the mask for the current site
      ext <- mask[which(mask$Site_Name == unique(mask$Site_Name)[site_i])] %>% 
        ext()
      
      resize_x <- (as.numeric(ext[2])- as.numeric(ext[1]))*0.2
      resize_y <- (as.numeric(ext[4])- as.numeric(ext[3]))*0.15

    plot <- ggplot()+
      geom_spatraster_rgb(data = rgb,
                          max_col_value = 0.8)+
      geom_tile(data = pred, aes(x = x, y = y, fill = SPC20Unknown), show.legend = F) +  # Use a continuous color scale and make NA values transparent
      coord_sf(xlim = c(ext[1]-resize_x, ext[2]+resize_x),
              ylim = c(ext[3]-resize_y, ext[4]+resize_y)) +
      scale_fill_viridis_c(na.value = "transparent") 
    
    if(site_i == 1){
      a <- plot
    }else{
      b = plot +
          scale_y_continuous(position = "left")
    }

      
      
  }
  
}

a+b